In [1]:
import gurobipy as gp
from gurobipy import GRB
import random as rd

In [2]:
A = [0, 1, 2,3,4,5,6]
T = [0,1,2]
V = [ 'walk',  'car']
shared = [False, False, True]
Available = [1000, 2, 2]

In [3]:
Q = 15
Budget = 10
n_passengers = 1
cost_rate_v = {'walk': 0, 'bike': 1.0, 'car': 2.0}
travel_time_v = {'walk': 1, 'bike': 0.5, 'car': 0.25}
fixed_cost = 5

distance_ij = {
    (0, 0): 0,  (0, 1): 4,  (0, 2): 8,  (0, 3): 6,  (0, 4): 7,  (0, 5): 3,  (0, 6): 5,  (0, 7): 9,  (0, 8): 2,  (0, 9): 6,
    (1, 0): 4,  (1, 1): 0,  (1, 2): 5,  (1, 3): 7,  (1, 4): 3,  (1, 5): 4,  (1, 6): 6,  (1, 7): 2,  (1, 8): 5,  (1, 9): 8,
    (2, 0): 8,  (2, 1): 5,  (2, 2): 0,  (2, 3): 4,  (2, 4): 6,  (2, 5): 7,  (2, 6): 5,  (2, 7): 3,  (2, 8): 6,  (2, 9): 9,
    (3, 0): 6,  (3, 1): 7,  (3, 2): 4,  (3, 3): 0,  (3, 4): 5,  (3, 5): 6,  (3, 6): 4,  (3, 7): 5,  (3, 8): 7,  (3, 9): 8,
    (4, 0): 7,  (4, 1): 3,  (4, 2): 6,  (4, 3): 5,  (4, 4): 0,  (4, 5): 4,  (4, 6): 3,  (4, 7): 6,  (4, 8): 5,  (4, 9): 7,
    (5, 0): 3,  (5, 1): 4,  (5, 2): 7,  (5, 3): 6,  (5, 4): 4,  (5, 5): 0,  (5, 6): 2,  (5, 7): 5,  (5, 8): 3,  (5, 9): 6,
    (6, 0): 5,  (6, 1): 6,  (6, 2): 5,  (6, 3): 4,  (6, 4): 3,  (6, 5): 2,  (6, 6): 0,  (6, 7): 4,  (6, 8): 3,  (6, 9): 5,
    (7, 0): 9,  (7, 1): 2,  (7, 2): 3,  (7, 3): 5,  (7, 4): 6,  (7, 5): 5,  (7, 6): 4,  (7, 7): 0,  (7, 8): 4,  (7, 9): 6,
    (8, 0): 2,  (8, 1): 5,  (8, 2): 6,  (8, 3): 7,  (8, 4): 5,  (8, 5): 3,  (8, 6): 3,  (8, 7): 4,  (8, 8): 0,  (8, 9): 5,
    (9, 0): 6,  (9, 1): 8,  (9, 2): 9,  (9, 3): 8,  (9, 4): 7,  (9, 5): 6,  (9, 6): 5,  (9, 7): 6,  (9, 8): 5,  (9, 9): 0,
}


t_ij_v = {}
for v, time_per_unit in travel_time_v.items():
    for i in A:
       for j in A:
            t_ij_v[v, i, j] = time_per_unit * distance_ij.get((i, j), 0)

print(t_ij_v)

tv_j = {
    0: 0,   # Depot usually has 0 service time
    1: 1,
    2: 2,
    3: 1,
    4: 1.5,
    5: 2,
    6: 1,
    7: 1.5,
    8: 1,
    9: 2
}


open_windows = {
    0: (8, 18),   # Depot: Open all day
    1: (9, 12),
    2: (10, 14),
    3: (11, 15),
    4: (13, 17),
    5: (9, 16),
    6: (10, 13),
    7: (14, 18),
    8: (9, 11),
    9: (15, 19)
}

for i in A:
    print(f"open_hours({i}) = {open_windows[i][0]} to {open_windows[i][1]}")


{('walk', 0, 0): 0, ('walk', 0, 1): 4, ('walk', 0, 2): 8, ('walk', 0, 3): 6, ('walk', 0, 4): 7, ('walk', 0, 5): 3, ('walk', 0, 6): 5, ('walk', 1, 0): 4, ('walk', 1, 1): 0, ('walk', 1, 2): 5, ('walk', 1, 3): 7, ('walk', 1, 4): 3, ('walk', 1, 5): 4, ('walk', 1, 6): 6, ('walk', 2, 0): 8, ('walk', 2, 1): 5, ('walk', 2, 2): 0, ('walk', 2, 3): 4, ('walk', 2, 4): 6, ('walk', 2, 5): 7, ('walk', 2, 6): 5, ('walk', 3, 0): 6, ('walk', 3, 1): 7, ('walk', 3, 2): 4, ('walk', 3, 3): 0, ('walk', 3, 4): 5, ('walk', 3, 5): 6, ('walk', 3, 6): 4, ('walk', 4, 0): 7, ('walk', 4, 1): 3, ('walk', 4, 2): 6, ('walk', 4, 3): 5, ('walk', 4, 4): 0, ('walk', 4, 5): 4, ('walk', 4, 6): 3, ('walk', 5, 0): 3, ('walk', 5, 1): 4, ('walk', 5, 2): 7, ('walk', 5, 3): 6, ('walk', 5, 4): 4, ('walk', 5, 5): 0, ('walk', 5, 6): 2, ('walk', 6, 0): 5, ('walk', 6, 1): 6, ('walk', 6, 2): 5, ('walk', 6, 3): 4, ('walk', 6, 4): 3, ('walk', 6, 5): 2, ('walk', 6, 6): 0, ('bike', 0, 0): 0.0, ('bike', 0, 1): 2.0, ('bike', 0, 2): 4.0, ('bik

In [4]:
m = gp.Model("model")
# Variables
x = m.addVars(A, A, V, T, vtype=GRB.BINARY, name="x")

m.setObjective(gp.quicksum(x[i, j, v, t]  for i in A for j in A for v in V for t in T), GRB.MAXIMIZE) 
# Constraints

m.addConstrs(gp.quicksum(x[i, i, v, t] for t in T for v in V) == 0 for i in A)
m.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for v in V) == gp.quicksum(x[j, i, v, t] for i in A for v in V) for j in A for t in T)
m.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for t in T for v in V) <= 1 for j in A[1:])

m.addConstrs(gp.quicksum(x[0, j, v, t] for j in A[1:] for v in V) <= 1 for t in T )

m.addConstrs(gp.quicksum(x[i, j, v, t] * (tv_j[j] + t_ij_v[v, i, j]) for i in A for j in A for v in V ) <= Q for t in T )

#Miller-Tucker-Zemlin formulation Subtour Elimination Constraints
q = [rd.randint(3, 3) for i in range(len(A)-1)]  # generate the random vector
u = m.addVars([i for i in range(len(A)-1)],V, vtype=GRB.CONTINUOUS, name="u")
for v in V: u[0, v].LB = 0
for v in V: u[0, v].UB = 0
P = [i for i in range(len(A)-1)]
for v in V :
    for i in P :
            
    #             u[i].LB = q[i]
        u[i, v].LB = min([(q[i]+(tv_j[i] + t_ij_v[v, j,i])) for j in P for t in T])
        u[i, v].UB = Q
    #     print([(q[i]+tm[j][i])* xijt[i,j,t] for i in P for j in P for t in T])
        m.addConstrs( u[i, v] - u[j, v] + Q * x[i, j, v, t] <= Q - q[j] for i in P for j in P for t in T if j != 0 )



Restricted license - for non-production use only - expires 2026-11-23


In [5]:
m.optimize()
if m.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for v in V:
        for i in A:
            for j in A:
                for t in T:
                    if x[i, j, v, t].x > 0.5:
                        print(f"Vehicle {v} travels from {i} to {j} at day {t}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1120 rows, 306 columns and 4002 nonzeros
Model fingerprint: 0xcab81d84
Variable types: 12 continuous, 294 integer (294 binary)
Coefficient statistics:
  Matrix range     [8e-01, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 957 rows and 77 columns
Presolve time: 0.02s
Presolved: 163 rows, 229 columns, 1255 nonzeros
Variable types: 10 continuous, 219 integer (219 binary)

Root relaxation: objective 9.000000e+00, 47 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

In [6]:
m1 = gp.Model("model tw")
# Variables
x = m1.addVars(A, A, V, T, vtype=GRB.BINARY, name="x")
t_ = m1.addVars(A,V, T, vtype=GRB.CONTINUOUS, name="t")
y = m1.addVars(T, vtype=GRB.BINARY, name='y')
b = m1.addVars(T, vtype=GRB.CONTINUOUS, name='b')



m1.setObjective(gp.quicksum(x[i, j, v, t]*(1/t_ij_v[v,i,j])  for i in A for j in A for v in V for t in T if i!=j), GRB.MAXIMIZE) 
# Constraints

m1.addConstrs(gp.quicksum(x[i, i, v, t] for t in T for v in V) == 0 for i in A)
m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for v in V) == gp.quicksum(x[j, i, v, t] for i in A for v in V) for j in A for t in T)
m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for t in T for v in V) <= 1 for j in A[1:])
m1.addConstrs(gp.quicksum(x[0, j, v, t] for j in A[1:] for v in V) == y[t] for t in T)
m1.addConstrs(gp.quicksum(x[i, j, v, t] * (tv_j[j] + t_ij_v[v, i, j]) for i in A for j in A for v in V ) <= Q for t in T )


m1.addConstrs(y[t] <= y[t+1] for t in T[:-1])  # Ensure y is non-decreasing
m1.addConstr(y[0]==1)  # Ensure y is non-increasing
# Time window constraints

m1.addConstrs(t_[i, v, t] >= open_windows[i][0] for i in A for v in V for t in T)
m1.addConstrs(t_[i, v, t] <= open_windows[i][1] for i in A for v in V for t in T)

#dialy budget constraints
m1.addConstrs(gp.quicksum((n_passengers if shared[V.index(v)] == False else 1)*x[i, j, v, t] * cost_rate_v[v]* t_ij_v[v, i, j] for i in A for j in A for v in V ) <= b[t] for t in T)
m1.addConstr(gp.quicksum(b[t] for t in T) + fixed_cost<= Budget)

#m1.addConstrs(gp.quicksum(x[i, j, v, t] for i in A for j in A) <= Available[V.index(v)] for v in V for t in T)

m1.addConstrs( x[i, j, v, t] * n_passengers  <= Available[V.index(v)] for i in A for j in A for v in V for t in T)

m1.addConstrs(t_[i, v, t] + tv_j[i] + t_ij_v[v, i, j] - 1000 * (1 - x[i, j, v, t]) <= t_[j, v, t] for v in V for i in A for j in A[1:] for t in T   )

#Miller-Tucker-Zemlin formulation Subtour Elimination Constraints
q = [rd.randint(3, 3) for i in range(len(A)-1)]  # generate the random vector
u = m1.addVars([i for i in range(len(A)-1)],V, vtype=GRB.CONTINUOUS, name="u")
for v in V: u[0, v].LB = 0
for v in V: u[0, v].UB = 0
P = [i for i in range(len(A)-1)]
for v in V :
    for i in P :
            
    #             u[i].LB = q[i]
        u[i, v].LB = min([(q[i]+(tv_j[i] + t_ij_v[v, j,i])) for j in P for t in T])
        u[i, v].UB = Q
    #     print([(q[i]+tm[j][i])* xijt[i,j,t] for i in P for j in P for t in T])
        m1.addConstrs( u[i, v] - u[j, v] + Q * x[i, j, v, t] <= Q - q[j] for i in P for j in P for t in T if j != 0 )


In [7]:
m1.optimize()
if m1.status == GRB.OPTIMAL:
    print("Optimal solution found")
    for t in T:
        for i in A:
            for j in A:
                for v in V:
                    if x[i, j, v, t].x > 0.5:
                        print(f"Vehicle {v} travels from {i} to {j} at day {t}")
                        print(f"Cost for vehicle {v} is {(n_passengers if shared[V.index(v)] == False else 1)*cost_rate_v[v] * t_ij_v[v, i, j]}")
                        print(f"Time at {i} is {t_[i, v, t].x}")
                        print(f"Time at {j} is {t_[j, v, t].x}")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1757 rows, 354 columns and 5204 nonzeros
Model fingerprint: 0xdae30075
Variable types: 57 continuous, 297 integer (297 binary)
Coefficient statistics:
  Matrix range     [8e-01, 1e+03]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 3.0000000
Presolve removed 1611 rows and 185 columns
Presolve time: 0.02s
Presolved: 146 rows, 169 columns, 1624 nonzeros
Variable types: 22 continuous, 147 integer (147 binary)

Root relaxation: objective 6.452381e+00, 78 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd  